In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# 🏦 PROYECTO: EXPANSIÓN Y LIMPIEZA DE DATASET BANCARIO
# Autor: Tu nombre
# Objetivo: Expandir dataset de 5,300 → 50,000 filas, ensuciar y subir a Firebase
# ═══════════════════════════════════════════════════════════════════════════

# ─────────────────────────────────────────────────────────────────────────
# PASO 1: INSTALACIÓN E IMPORTACIÓN DE LIBRERÍAS
# ─────────────────────────────────────────────────────────────────────────
!pip install kagglehub -q

import kagglehub
import pandas as pd
import numpy as np
import uuid
import requests
import json
import os
from datetime import datetime, timedelta
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías cargadas correctamente\n")

# ─────────────────────────────────────────────────────────────────────────
# PASO 2: DESCARGA Y CARGA DEL DATASET ORIGINAL
# ─────────────────────────────────────────────────────────────────────────
print("📥 Descargando dataset de Kaggle...")
try:
    path = kagglehub.dataset_download("pradeepkumar2424/usa-banking-transactions-dataset-2023-2024")
    print(f"✅ Dataset descargado en: {path}")

    # Buscar archivo CSV
    archivos = os.listdir(path)
    csv_files = [f for f in archivos if f.endswith('.csv')]

    if not csv_files:
        raise FileNotFoundError("❌ No se encontró ningún archivo CSV")

    # Cargar el DataFrame
    full_path = os.path.join(path, csv_files[0])
    df_inicio = pd.read_csv(full_path)

    print(f"✅ Archivo cargado: {csv_files[0]}")
    print(f"📊 Dimensiones originales: {df_inicio.shape}")
    print(f"📋 Columnas: {list(df_inicio.columns)}\n")

except Exception as e:
    print(f"❌ Error al cargar dataset: {e}")
    raise

# # ─────────────────────────────────────────────────────────────────────────
# PASO 3: EXPANSIÓN DEL DATASET (5,300 → 50,000 FILAS)
# ─────────────────────────────────────────────────────────────────────────
print("🔄 Expandiendo dataset a 50,000 filas...")

OBJETIVO_FILAS = 50000
filas_actuales = len(df_inicio)
filas_a_crear = OBJETIVO_FILAS - filas_actuales

print(f"   • Filas actuales: {filas_actuales:,}")
print(f"   • Filas a generar: {filas_a_crear:,}")

# Muestreo con reemplazo
df_nuevos = df_inicio.sample(n=filas_a_crear, replace=True).copy()

# Generar fechas aleatorias entre 2020-2022
def random_dates(start_year, end_year, n):
    start = datetime(start_year, 1, 1)
    end = datetime(end_year, 12, 31, 23, 59, 59)
    delta = end - start
    int_deltas = np.random.randint(0, int(delta.total_seconds()), n)
    return [start + timedelta(seconds=int(i)) for i in int_deltas]

fechas_generadas = random_dates(2020, 2022, filas_a_crear)
df_nuevos['Transaction_Date'] = fechas_generadas

# CORRECCIÓN: Regenerar IDs únicos GARANTIZADOS
print("   • Generando IDs únicos...")
ids_existentes = set(df_inicio['Transaction_ID'].astype(str))
nuevos_ids = []

for _ in range(filas_a_crear):
    nuevo_id = str(uuid.uuid4())
    while nuevo_id in ids_existentes:  # Evita colisiones (extremadamente raro pero posible)
        nuevo_id = str(uuid.uuid4())
    nuevos_ids.append(nuevo_id)
    ids_existentes.add(nuevo_id)

df_nuevos['Transaction_ID'] = nuevos_ids

# También regenerar Account_Numbers con variación
if 'Account_Number' in df_nuevos.columns:
    account_base = df_nuevos['Account_Number'].astype(str)
    df_nuevos['Account_Number'] = account_base.apply(
        lambda x: str(int(x) + np.random.randint(-1000, 1000)) if x.isdigit() else x
    )

# Unir datasets
df_completo = pd.concat([df_inicio, df_nuevos], ignore_index=True)
df_completo['Transaction_Date'] = pd.to_datetime(df_completo['Transaction_Date'])

print(f"✅ Dataset expandido: {df_completo.shape}")
print(f"   • Rango de fechas: {df_completo['Transaction_Date'].min().date()} a {df_completo['Transaction_Date'].max().date()}")
print(f"   • IDs únicos verificados: {df_completo['Transaction_ID'].nunique() == len(df_completo)}\n")

# ─────────────────────────────────────────────────────────────────────────
# PASO 4: ENSUCIAMIENTO INTELIGENTE DEL DATASET (KDD)
# ─────────────────────────────────────────────────────────────────────────
print("🧹 Ensuciando datos para proceso KDD...")

df = df_completo.copy()

# 4.1 - VALORES NULOS (5% en columnas realistas)
print("   • Insertando valores nulos (5%)...")
columnas_nulos = ['Customer_Occupation', 'Transaction_Description', 'City']
for col in columnas_nulos:
    if col in df.columns:
        mask_nulos = np.random.rand(len(df)) < 0.05
        df.loc[mask_nulos, col] = np.nan

nulos_totales = df.isnull().sum().sum()
print(f"     → {nulos_totales:,} valores nulos insertados")

# 4.2 - INCONSISTENCIAS DE TEXTO (7%)
print("   • Insertando inconsistencias de texto (7%)...")
if 'City' in df.columns:
    indices_ciudad = df.sample(frac=0.07).index
    df.loc[indices_ciudad, 'City'] = df.loc[indices_ciudad, 'City'].str.lower()

if 'Payment_Method' in df.columns:
    indices_pago = df.sample(frac=0.07).index
    df.loc[indices_pago, 'Payment_Method'] = df.loc[indices_pago, 'Payment_Method'] + "  "

inconsistencias = len(indices_ciudad) + len(indices_pago)
print(f"     → {inconsistencias:,} inconsistencias insertadas")

# 4.3 - OUTLIERS EN MONTOS (25 transacciones extremas)
print("   • Insertando outliers en montos (25)...")
if 'Transaction_Amount' in df.columns:
    indices_outliers = df.sample(n=25).index
    df.loc[indices_outliers, 'Transaction_Amount'] = df.loc[indices_outliers, 'Transaction_Amount'] * np.random.uniform(50, 150, 25)
    print(f"     → 25 outliers insertados")

# 4.4 - DUPLICADOS (100 filas con IDs NUEVOS pero contenido duplicado)
print("   • Insertando duplicados (100)...")
filas_duplicadas = df.sample(n=100).copy()

# IMPORTANTE: Generar nuevos IDs para los duplicados (para evitar error de índice único)
nuevos_ids_duplicados = []
for _ in range(100):
    nuevo_id = str(uuid.uuid4())
    while nuevo_id in ids_existentes:
        nuevo_id = str(uuid.uuid4())
    nuevos_ids_duplicados.append(nuevo_id)
    ids_existentes.add(nuevo_id)

filas_duplicadas['Transaction_ID'] = nuevos_ids_duplicados

df = pd.concat([df, filas_duplicadas], ignore_index=True)

# Ahora verificamos duplicados SIN considerar Transaction_ID
columnas_sin_id = [col for col in df.columns if col != 'Transaction_ID']
duplicados_contenido = df.duplicated(subset=columnas_sin_id, keep=False).sum()
print(f"     → {duplicados_contenido:,} filas con contenido duplicado")

# Mezclar dataset
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"\n✅ Dataset ensuciado completamente")
print(f"   📊 Dimensiones finales: {df.shape}")
print(f"   🔍 Resumen de suciedad:")
print(f"      - Valores nulos: {df.isnull().sum().sum():,}")
print(f"      - Duplicados (contenido): {duplicados_contenido:,}")
print(f"      - Outliers: 25 transacciones")
print(f"      - IDs únicos: {df['Transaction_ID'].nunique() == len(df)} ✅\n")

# ─────────────────────────────────────────────────────────────────────────
# PASO 5: PREPARACIÓN PARA FIREBASE
# ─────────────────────────────────────────────────────────────────────────
print("🔧 Preparando datos para Firebase...")

# Convertir fechas a string
df_subida = df.copy()
df_subida['Transaction_Date'] = df_subida['Transaction_Date'].astype(str)

# Rellenar NaN para evitar errores en JSON
df_subida = df_subida.fillna("N/A")

# VERIFICACIÓN FINAL: Asegurar que todos los IDs son únicos
if df_subida['Transaction_ID'].duplicated().any():
    print("⚠️  ADVERTENCIA: Se encontraron IDs duplicados, regenerando...")
    duplicados_mask = df_subida['Transaction_ID'].duplicated(keep='first')
    num_duplicados = duplicados_mask.sum()

    for idx in df_subida[duplicados_mask].index:
        nuevo_id = str(uuid.uuid4())
        while nuevo_id in df_subida['Transaction_ID'].values:
            nuevo_id = str(uuid.uuid4())
        df_subida.at[idx, 'Transaction_ID'] = nuevo_id

    print(f"✅ {num_duplicados} IDs regenerados correctamente")

# Convertir a diccionario usando Transaction_ID como clave
data_firebase = df_subida.set_index('Transaction_ID').to_dict(orient='index')

print(f"✅ {len(data_firebase):,} registros preparados para subida")
print(f"   • Verificación final: {len(data_firebase) == len(df_subida)} ✅\n")

# ─────────────────────────────────────────────────────────────────────────
# PASO 6: SUBIDA A FIREBASE (EN LOTES OPTIMIZADOS)
# ─────────────────────────────────────────────────────────────────────────
print("☁️  Subiendo a Firebase Realtime Database...")

BASE_URL = "https://bifrost-e5592-default-rtdb.firebaseio.com/"
NODO = "transacciones_bancarias"
TAMAÑO_LOTE = 1000  # Firebase maneja bien lotes de ~1000 registros

def subir_lote(nodo, datos_lote, lote_num):
    """Sube un lote de datos a Firebase"""
    url = f"{BASE_URL}{nodo}.json"
    try:
        response = requests.patch(url, data=json.dumps(datos_lote), timeout=30)
        if response.status_code == 200:
            return True, None
        else:
            return False, response.text
    except Exception as e:
        return False, str(e)

# Dividir en lotes
transaction_ids = list(data_firebase.keys())
total_lotes = (len(transaction_ids) + TAMAÑO_LOTE - 1) // TAMAÑO_LOTE

print(f"   • Total de lotes: {total_lotes}")
print(f"   • Tamaño por lote: ~{TAMAÑO_LOTE} registros\n")

exitosos = 0
fallidos = 0

for i in tqdm(range(total_lotes), desc="Subiendo lotes"):
    inicio = i * TAMAÑO_LOTE
    fin = min((i + 1) * TAMAÑO_LOTE, len(transaction_ids))

    ids_lote = transaction_ids[inicio:fin]
    datos_lote = {id_: data_firebase[id_] for id_ in ids_lote}

    exito, error = subir_lote(NODO, datos_lote, i+1)

    if exito:
        exitosos += len(datos_lote)
    else:
        fallidos += len(datos_lote)
        print(f"\n⚠️  Error en lote {i+1}: {error}")

print(f"\n{'='*60}")
print(f"✅ SUBIDA COMPLETADA")
print(f"{'='*60}")
print(f"   • Registros exitosos: {exitosos:,}")
print(f"   • Registros fallidos: {fallidos:,}")
print(f"   • URL Firebase: {BASE_URL}{NODO}")
print(f"\n🎉 ¡Proceso terminado! Tu dataset está listo para análisis KDD.\n")

# ─────────────────────────────────────────────────────────────────────────
# BONUS: VERIFICACIÓN RÁPIDA
# ─────────────────────────────────────────────────────────────────────────
print("🔍 Verificación rápida de datos subidos...")
try:
    response = requests.get(f"{BASE_URL}{NODO}.json?shallow=true&limitToFirst=1")
    if response.status_code == 200:
        data = response.json()
        if data:
            print(f"✅ Firebase respondiendo correctamente")
            print(f"   • Primer ID en base: {list(data.keys())[0]}")
        else:
            print("⚠️  Advertencia: Firebase vacío (puede tardar unos segundos en sincronizar)")
except Exception as e:
    print(f"⚠️  No se pudo verificar: {e}")

print("\n" + "="*60)
print("📝 NOTAS FINALES:")
print("="*60)
print("1. Dataset expandido de 5,389 → 50,100 filas")
print("2. Datos ensuciados listos para limpieza KDD")
print("3. Subido a Firebase en un único nodo: 'transacciones_bancarias'")
print("4. Para limpiar, busca: nulos, duplicados, outliers, inconsistencias")
print("5. Todos los Transaction_IDs son únicos (sin colisiones)")
print("\n¡Buena suerte con tu proyecto! 🚀")

✅ Librerías cargadas correctamente

📥 Descargando dataset de Kaggle...


100%|██████████| 580k/580k [00:00<00:00, 64.0MB/s]

Extracting files...
✅ Dataset descargado en: /root/.cache/kagglehub/datasets/pradeepkumar2424/usa-banking-transactions-dataset-2023-2024/versions/1
✅ Archivo cargado: Banking_Transactions_USA_2023_2024.csv
📊 Dimensiones originales: (5389, 20)
📋 Columnas: ['Transaction_ID', 'Account_Number', 'Transaction_Date', 'Transaction_Amount', 'Merchant_Name', 'Transaction_Type', 'Category', 'City', 'Country', 'Payment_Method', 'Customer_Age', 'Customer_Gender', 'Customer_Occupation', 'Customer_Income', 'Account_Balance', 'Transaction_Status', 'Fraud_Flag', 'Discount_Applied', 'Loyalty_Points_Earned', 'Transaction_Description']

🔄 Expandiendo dataset a 50,000 filas...
   • Filas actuales: 5,389
   • Filas a generar: 44,611


   • Generando IDs únicos...
✅ Dataset expandido: (50000, 20)
   • Rango de fechas: 2020-01-01 a 2025-01-20
   • IDs únicos verificados: True

🧹 Ensuciando datos para proceso KDD...
   • Insertando valores nulos (5%)...
     → 7,443 valores nulos insertados
   • Insertando inconsistencias de texto (7%)...
     → 7,000 inconsistencias insertadas
   • Insertando outliers en montos (25)...
     → 25 outliers insertados
   • Insertando duplicados (100)...
     → 200 filas con contenido duplicado

✅ Dataset ensuciado completamente
   📊 Dimensiones finales: (50100, 20)
   🔍 Resumen de suciedad:
      - Valores nulos: 7,464
      - Duplicados (contenido): 200
      - Outliers: 25 transacciones
      - IDs únicos: True ✅

🔧 Preparando datos para Firebase...
✅ 50,100 registros preparados para subida
   • Verificación final: True ✅

☁️  Subiendo a Firebase Realtime Database...
   • Total de lotes: 51
   • Tamaño por lote: ~1000 registros



Subiendo lotes: 100%|██████████| 51/51 [00:26<00:00,  1.93it/s]



✅ SUBIDA COMPLETADA
   • Registros exitosos: 50,100
   • Registros fallidos: 0
   • URL Firebase: https://bifrost-e5592-default-rtdb.firebaseio.com/transacciones_bancarias

🎉 ¡Proceso terminado! Tu dataset está listo para análisis KDD.

🔍 Verificación rápida de datos subidos...
✅ Firebase respondiendo correctamente
   • Primer ID en base: 000030fb-cbf0-42ab-8719-0c7862e73f7f

📝 NOTAS FINALES:
1. Dataset expandido de 5,389 → 50,100 filas
2. Datos ensuciados listos para limpieza KDD
3. Subido a Firebase en un único nodo: 'transacciones_bancarias'
4. Para limpiar, busca: nulos, duplicados, outliers, inconsistencias
5. Todos los Transaction_IDs son únicos (sin colisiones)

¡Buena suerte con tu proyecto! 🚀


In [ ]:
df

,Transaction_ID,Account_Number,Transaction_Date,Transaction_Amount,Merchant_Name,Transaction_Type,Category,City,Country,Payment_Method,Customer_Age,Customer_Gender,Customer_Occupation,Customer_Income,Account_Balance,Transaction_Status,Fraud_Flag,Discount_Applied,Loyalty_Points_Earned,Transaction_Description
0,65cdd7b5-80cb-44e2-badd-8bdffeb1f48e,RPZD55039303803543,2022-01-24 18:26:07,3339.00,"Rodriguez, Perez and Davis",Debit,Food,Los Angeles,USA,Online Transfer,36,Others,Cytogeneticist,25423.57,8402.20,Failed,Yes,False,54,Soldier town control serve however adult near.
1,cc2b2f49-c6d5-4a90-9296-ae08a2dcb574,VHWV64903657168897,2024-03-03 09:18:37,1152.92,Gomez PLC,Debit,Electronics,Chicago,USA,Credit Card,64,Others,Manufacturing systems engineer,61259.56,9965.04,Success,No,True,425,Six bed simple prove compare against scientist...
2,368af1a6-70f1-457d-ab1b-d6fd46ce10ad,LVBO11383514635476,2021-12-28 19:29:34,1832.09,"Ramos, Lopez and Robinson",Credit,Housing,Los Angeles,USA,Cash,68,Female,Passenger transport manager,102988.65,18545.48,Pending,Yes,False,425,Live I by help wish.
3,df391d94-a2de-43e5-be99-6e239e2a577e,BRUY69227025254110,2022-05-14 14:44:09,1163.12,"Lee, Jackson and Jackson",Debit,Utilities,Chicago,USA,Cash,47,Female,Environmental consultant,46604.80,3250.94,Success,No,False,447,Attention worker short tonight.
4,dbd7dd5d-3457-4fee-887d-7e427c5fb9ef,IOZZ48485694451421,2022-12-16 03:13:31,3661.27,"Williams, Fry and Chandler",Debit,Savings,San Jose,USA,Debit Card,39,Female,Investment analyst,28624.64,465.89,Success,No,True,367,Upon must black college.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50095,bec5b162-994c-44b0-b9f9-bdf8f3a6f1bf,WQBP66487841383127,2021-02-24 23:10:43,374.46,"Graham, Bolton and Jennings",Credit,Savings,Houston,USA,Cash,48,Female,Forest/woodland manager,23389.02,3975.51,Success,No,False,67,Answer act imagine everyone time none.
50096,f67ed246-16d4-4a0a-bfae-78cf1fc2922e,BLST44617353606844,2021-08-22 05:41:28,3472.49,Phillips Group,Credit,Fitness,NaN,USA,Cash,58,Others,IT sales professional,118343.82,8917.96,Success,Yes,True,353,Fast traditional risk white she we reality.
50097,e4cf868d-2993-4d06-ab14-7a63c26c7ab5,XYTA78611407670347,2021-06-15 02:39:03,2061.62,Phelps PLC,Debit,Housing,Phoenix,USA,Online Transfer,48,Male,Lexicographer,96914.21,1042.69,Pending,No,True,117,Same himself worker human.
50098,98cf1188-06d2-4913-a117-3719b023a0ea,GDED46394762542926,2023-02-02 05:03:38,4363.53,Lopez PLC,Credit,Savings,Houston,USA,Debit Card,55,Female,Lobbyist,60156.97,4953.27,Success,Yes,False,427,Community war actually prove get.


In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# 🏦 PROYECTO: GENERACIÓN DE DATASET BANCARIO (DISTRIBUCIÓN EQUILIBRADA)
# ═══════════════════════════════════════════════════════════════════════════

import kagglehub
import pandas as pd
import numpy as np
import uuid
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✅ Entorno configurado.\n")

# ─────────────────────────────────────────────────────────────────────────
# PASO 2: EXTRACCIÓN Y LIMPIEZA ESTRUCTURAL
# ─────────────────────────────────────────────────────────────────────────
print("📥 Obteniendo esquema de datos...")
try:
    path = kagglehub.dataset_download("pradeepkumar2424/usa-banking-transactions-dataset-2023-2024")
    csv_files = [f for f in os.listdir(path) if f.endswith('.csv')]

    if not csv_files: raise FileNotFoundError("CSV no encontrado.")

    # Cargamos solo para obtener estructura y valores categóricos válidos
    df_origen = pd.read_csv(os.path.join(path, csv_files[0]))

    # Creamos una plantilla limpia (sin filas)
    print(f"✅ Esquema detectado: {df_origen.columns.tolist()}")

except Exception as e:
    print(f"❌ Error crítico: {e}")
    raise

# ─────────────────────────────────────────────────────────────────────────
# PASO 3: ALGORITMO DE GENERACIÓN CONTROLADA (MAX 15K/AÑO)
# ─────────────────────────────────────────────────────────────────────────
OBJETIVO_FILAS = 50000
print(f"🔄 Generando {OBJETIVO_FILAS:,} transacciones con distribución acotada...")

# Usamos muestreo con reemplazo del original para mantener coherencia en datos categóricos
# pero reiniciaremos las fechas y IDs totalmente.
df_generado = df_origen.sample(n=OBJETIVO_FILAS, replace=True).copy()

def generate_balanced_dates(n):
    """
    Distribuye fechas respetando un límite de ~15k por año.
    Patrón: Alto -> Bajo -> Max(15k) -> Bajo -> Alto
    """
    # DEFINICIÓN DE PESOS (Suma = 1.0)
    # 0.30 * 50,000 = 15,000 (Cumple la restricción exacta del pico)
    scenario_probs = {
        2020: 0.20,  # ~10,000
        2021: 0.15,  # ~7,500  (Caída respecto a 2020)
        2022: 0.30,  # ~15,000 (PICO MÁXIMO)
        2023: 0.15,  # ~7,500  (Valle)
        2024: 0.20   # ~10,000 (Recuperación)
    }

    years = list(scenario_probs.keys())
    probs = list(scenario_probs.values())

    # 1. Asignación vectorizada de años
    chosen_years = np.random.choice(years, size=n, p=probs)

    # 2. Generación de fechas intra-anuales
    dates = []
    counts = pd.Series(chosen_years).value_counts()

    for year, count in counts.items():
        # Timestamp de inicio y fin de año
        start = pd.Timestamp(f"{year}-01-01").value
        end = pd.Timestamp(f"{year}-12-31").value

        # Generación uniforme dentro del año
        random_ts = np.random.randint(start, end, count)
        dates.extend(pd.to_datetime(random_ts))

    np.random.shuffle(dates)
    return dates

# Aplicar generador
fechas_finales = generate_balanced_dates(OBJETIVO_FILAS)
df_generado['Transaction_Date'] = fechas_finales

# 3.1 Integridad Referencial (Nuevos IDs)
print("   • Asignando UUIDs v4 únicos...")
df_generado['Transaction_ID'] = [str(uuid.uuid4()) for _ in range(OBJETIVO_FILAS)]

# 3.2 Variación de Cuentas (Noise Injection)
if 'Account_Number' in df_generado.columns:
    df_generado['Account_Number'] = df_generado['Account_Number'].apply(
        lambda x: str(int(str(x)) + np.random.randint(-500, 500)) if str(x).isdigit() else x
    )

# Ordenamiento temporal final
df_final = df_generado.sort_values('Transaction_Date').reset_index(drop=True)

# ─────────────────────────────────────────────────────────────────────────
# PASO 4: ENSUCIAMIENTO DE DATOS (KDD PREPARATION)
# ─────────────────────────────────────────────────────────────────────────
print("🧹 Inyectando anomalías y ruido...")

df = df_final.copy()

# Nulos (5%)
columnas_sucias = ['Customer_Occupation', 'Transaction_Description', 'City']
for col in columnas_sucias:
    if col in df.columns:
        df.loc[df.sample(frac=0.05).index, col] = np.nan

# Inconsistencias de Texto (7%)
if 'City' in df.columns:
    idx = df.sample(frac=0.07).index
    df.loc[idx, 'City'] = df.loc[idx, 'City'].str.lower()

# Outliers (Montos)
if 'Transaction_Amount' in df.columns:
    # 20 transacciones con montos inflados
    idx_out = df.sample(n=20).index
    df.loc[idx_out, 'Transaction_Amount'] *= np.random.uniform(10, 50, 20)

print(f"✅ Dataset finalizado: {df.shape}")

# ─────────────────────────────────────────────────────────────────────────
# VALIDACIÓN DE REQUISITOS (PROOF OF DISTRIBUTION)
# ─────────────────────────────────────────────────────────────────────────
print("\n📊 REPORTE DE DISTRIBUCIÓN ANUAL:")
print("="*60)
conteo = df['Transaction_Date'].dt.year.value_counts().sort_index()
print(conteo)
print("="*60)
print(f"TOTAL: {conteo.sum():,}")

# Assertions para garantizar calidad antes de subir
pico = conteo.max()
anio_pico = conteo.idxmax()

try:
    assert pico <= 15500, f"⚠️ ALERTA: El pico ({pico}) excede el límite de tolerancia (15k)."
    assert anio_pico == 2022, f"⚠️ ALERTA: El año pico es {anio_pico}, debería ser 2022."
    assert conteo[2021] < conteo[2020], "⚠️ ALERTA: No hubo bajada en 2021."
    print("\n✅ VALIDACIÓN EXITOSA: Distribución correcta y bajo el límite de 15k.")
except AssertionError as e:
    print(f"\n❌ ERROR DE VALIDACIÓN: {e}")

# ─────────────────────────────────────────────────────────────────────────
# PASO 6: CARGA A FIREBASE (MODO SILENCIOSO / COMENTADO)
# ─────────────────────────────────────────────────────────────────────────
"""
print("\n☁️  Preparando subida a Firebase...")

# Serialización
df_subida = df.copy()
df_subida['Transaction_Date'] = df_subida['Transaction_Date'].astype(str)
df_subida = df_subida.fillna("N/A")
data_firebase = df_subida.set_index('Transaction_ID').to_dict(orient='index')

BASE_URL = "https://bifrost-e5592-default-rtdb.firebaseio.com/"
NODO = "transacciones_bancarias"
TAMAÑO_LOTE = 1000

# Lógica de subida (request.patch) omitida intencionalmente para revisión de datos.
# Descomentar el bucle de subida cuando la distribución de datos sea aprobada.
"""

print("\n🚀 LISTO. Revisa los conteos arriba. Si te gusta la distribución, descomenta la carga.")

✅ Entorno configurado.

📥 Obteniendo esquema de datos...
Using Colab cache for faster access to the 'usa-banking-transactions-dataset-2023-2024' dataset.
✅ Esquema detectado: ['Transaction_ID', 'Account_Number', 'Transaction_Date', 'Transaction_Amount', 'Merchant_Name', 'Transaction_Type', 'Category', 'City', 'Country', 'Payment_Method', 'Customer_Age', 'Customer_Gender', 'Customer_Occupation', 'Customer_Income', 'Account_Balance', 'Transaction_Status', 'Fraud_Flag', 'Discount_Applied', 'Loyalty_Points_Earned', 'Transaction_Description']
🔄 Generando 50,000 transacciones con distribución acotada...
   • Asignando UUIDs v4 únicos...
🧹 Inyectando anomalías y ruido...
✅ Dataset finalizado: (50000, 20)

📊 REPORTE DE DISTRIBUCIÓN ANUAL:
Transaction_Date
2020     9913
2021     7583
2022    15067
2023     7354
2024    10083
Name: count, dtype: int64
TOTAL: 50,000

✅ VALIDACIÓN EXITOSA: Distribución correcta y bajo el límite de 15k.

🚀 LISTO. Revisa los conteos arriba. Si te gusta la distribuci

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# 🏦 PROYECTO: GENERACIÓN DE DATASET BANCARIO (CRECIMIENTO REALISTA)
# Autor: Tu nombre
# Objetivo: Simular un banco en expansión (Tendencia positiva para predicción)
# ═══════════════════════════════════════════════════════════════════════════

import kagglehub
import pandas as pd
import numpy as np
import uuid
import os
import requests
import json
from datetime import datetime
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("✅ Entorno configurado.\n")

# ─────────────────────────────────────────────────────────────────────────
# PASO 2: EXTRACCIÓN Y LIMPIEZA ESTRUCTURAL
# ─────────────────────────────────────────────────────────────────────────
print("📥 Obteniendo esquema de datos...")
try:
    path = kagglehub.dataset_download("pradeepkumar2424/usa-banking-transactions-dataset-2023-2024")
    csv_files = [f for f in os.listdir(path) if f.endswith('.csv')]

    if not csv_files: raise FileNotFoundError("CSV no encontrado.")

    # Cargamos solo para obtener estructura
    df_origen = pd.read_csv(os.path.join(path, csv_files[0]))
    print(f"✅ Esquema detectado: {df_origen.columns.tolist()}")

except Exception as e:
    print(f"❌ Error crítico: {e}")
    raise

# ─────────────────────────────────────────────────────────────────────────
# PASO 3: ALGORITMO DE GENERACIÓN (CRECIMIENTO SOSTENIDO)
# ─────────────────────────────────────────────────────────────────────────
OBJETIVO_FILAS = 50000
print(f"🔄 Generando {OBJETIVO_FILAS:,} transacciones con TENDENCIA POSITIVA...")

# Muestreo base
df_generado = df_origen.sample(n=OBJETIVO_FILAS, replace=True).copy()

def generate_growth_dates(n):
    """
    Distribuye fechas simulan un NEGOCIO EN CRECIMIENTO.
    Cada año tiene más transacciones que el anterior.
    """
    # 📊 ESTRATEGIA: Crecimiento Lineal
    # La suma de probabilidades debe ser 1.0
    scenario_probs = {
        2020: 0.10,  # 10% (Inicio: ~5,000 tx)
        2021: 0.15,  # 15% (Crecimiento)
        2022: 0.20,  # 20% (Consolidación)
        2023: 0.25,  # 25% (Expansión)
        2024: 0.30   # 30% (Éxito actual: ~15,000 tx)
    }

    years = list(scenario_probs.keys())
    probs = list(scenario_probs.values())

    # 1. Asignación de años basada en probabilidad de crecimiento
    chosen_years = np.random.choice(years, size=n, p=probs)

    # 2. Generación de fechas detalladas
    dates = []
    counts = pd.Series(chosen_years).value_counts()

    for year, count in counts.items():
        start = pd.Timestamp(f"{year}-01-01").value
        end = pd.Timestamp(f"{year}-12-31").value
        # Uniforme dentro del año
        random_ts = np.random.randint(start, end, count)
        dates.extend(pd.to_datetime(random_ts))

    np.random.shuffle(dates)
    return dates

# Aplicar generador
fechas_finales = generate_growth_dates(OBJETIVO_FILAS)
df_generado['Transaction_Date'] = fechas_finales

# 3.1 Integridad Referencial
print("   • Asignando UUIDs v4 únicos...")
df_generado['Transaction_ID'] = [str(uuid.uuid4()) for _ in range(OBJETIVO_FILAS)]

# 3.2 Variación de Cuentas (Ruido leve)
if 'Account_Number' in df_generado.columns:
    df_generado['Account_Number'] = df_generado['Account_Number'].apply(
        lambda x: str(int(str(x)) + np.random.randint(-500, 500)) if str(x).isdigit() else x
    )

# Ordenar cronológicamente (ayuda a la visualización)
df_final = df_generado.sort_values('Transaction_Date').reset_index(drop=True)

# ─────────────────────────────────────────────────────────────────────────
# PASO 4: ENSUCIAMIENTO DE DATOS (PREPARACIÓN KDD)
# ─────────────────────────────────────────────────────────────────────────
print("🧹 Inyectando anomalías y ruido (Data Dirtying)...")

df = df_final.copy()

# Nulos (5%)
columnas_sucias = ['Customer_Occupation', 'Transaction_Description', 'City']
for col in columnas_sucias:
    if col in df.columns:
        df.loc[df.sample(frac=0.05).index, col] = np.nan

# Inconsistencias de Texto (7%) - "new york" vs "New York"
if 'City' in df.columns:
    idx = df.sample(frac=0.07).index
    df.loc[idx, 'City'] = df.loc[idx, 'City'].str.lower()

# Outliers (Montos extremos)
if 'Transaction_Amount' in df.columns:
    # 25 transacciones con montos multiplicados por 10x-50x
    idx_out = df.sample(n=25).index
    df.loc[idx_out, 'Transaction_Amount'] *= np.random.uniform(10, 50, 25)

print(f"✅ Dataset finalizado: {df.shape}")

# ─────────────────────────────────────────────────────────────────────────
# VALIDACIÓN DE REQUISITOS (PROOF OF GROWTH)
# ─────────────────────────────────────────────────────────────────────────
print("\n📊 REPORTE DE DISTRIBUCIÓN ANUAL (Debería ser creciente):")
print("="*60)
conteo = df['Transaction_Date'].dt.year.value_counts().sort_index()
print(conteo)
print("="*60)

# Validaciones actualizadas para Crecimiento
try:
    # Verificar que 2024 tiene más tráfico que 2020 (Tendencia Positiva)
    assert conteo[2024] > conteo[2020], "⚠️ ALERTA: No hay crecimiento a largo plazo."

    # Verificar que el último año es el más fuerte (o casi)
    assert conteo.idxmax() == 2024, f"⚠️ ALERTA: El año pico no es el actual ({conteo.idxmax()})."

    print("\n✅ VALIDACIÓN EXITOSA: El dataset muestra un crecimiento saludable.")
    print("   (Ideal para predecir tendencias futuras)")
except AssertionError as e:
    print(f"\n❌ ERROR DE VALIDACIÓN: {e}")

# ─────────────────────────────────────────────────────────────────────────
# PASO 6: CARGA A FIREBASE
# ─────────────────────────────────────────────────────────────────────────

# ⚠️ SI ESTÁS LISTO, DESCOMENTA ESTO PARA SUBIR LOS DATOS ⚠️

"""
print("\n☁️  Subiendo a Firebase Realtime Database...")

# Preparar datos
df_subida = df.copy()
df_subida['Transaction_Date'] = df_subida['Transaction_Date'].astype(str)
df_subida = df_subida.fillna("N/A")
data_firebase = df_subida.set_index('Transaction_ID').to_dict(orient='index')

BASE_URL = "https://bifrost-e5592-default-rtdb.firebaseio.com/" # TU URL AQUÍ
NODO = "transacciones_bancarias"
TAMAÑO_LOTE = 1000

def subir_lote(nodo, datos_lote):
    url = f"{BASE_URL}{nodo}.json"
    try:
        response = requests.patch(url, data=json.dumps(datos_lote), timeout=30)
        return response.status_code == 200
    except:
        return False

# Dividir y subir
transaction_ids = list(data_firebase.keys())
total_lotes = (len(transaction_ids) + TAMAÑO_LOTE - 1) // TAMAÑO_LOTE

exitosos = 0
for i in tqdm(range(total_lotes), desc="Subiendo lotes"):
    inicio = i * TAMAÑO_LOTE
    fin = min((i + 1) * TAMAÑO_LOTE, len(transaction_ids))
    ids_lote = transaction_ids[inicio:fin]
    datos_lote = {id_: data_firebase[id_] for id_ in ids_lote}

    if subir_lote(NODO, datos_lote):
        exitosos += len(datos_lote)

print(f"\n✅ SUBIDA COMPLETADA: {exitosos:,} registros subidos.")
"""

print("\n🚀 LISTO. Revisa que los años 2020->2024 vayan subiendo en cantidad.")

✅ Entorno configurado.

📥 Obteniendo esquema de datos...


100%|██████████| 580k/580k [00:00<00:00, 32.0MB/s]

Extracting files...


✅ Esquema detectado: ['Transaction_ID', 'Account_Number', 'Transaction_Date', 'Transaction_Amount', 'Merchant_Name', 'Transaction_Type', 'Category', 'City', 'Country', 'Payment_Method', 'Customer_Age', 'Customer_Gender', 'Customer_Occupation', 'Customer_Income', 'Account_Balance', 'Transaction_Status', 'Fraud_Flag', 'Discount_Applied', 'Loyalty_Points_Earned', 'Transaction_Description']
🔄 Generando 50,000 transacciones con TENDENCIA POSITIVA...
   • Asignando UUIDs v4 únicos...
🧹 Inyectando anomalías y ruido (Data Dirtying)...
✅ Dataset finalizado: (50000, 20)

📊 REPORTE DE DISTRIBUCIÓN ANUAL (Debería ser creciente):
Transaction_Date
2020     4906
2021     7786
2022    10021
2023    12464
2024    14823
Name: count, dtype: int64

✅ VALIDACIÓN EXITOSA: El dataset muestra un crecimiento saludable.
   (Ideal para predecir tendencias futuras)

🚀 LISTO. Revisa que los años 2020->2024 vayan subiendo en cantidad.


In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# 🏦 GENERADOR DEFINITIVO -> DATASET_BANCARIO_KDD_50K.CSV EN DRIVE
# ═══════════════════════════════════════════════════════════════════════════

import kagglehub
import pandas as pd
import numpy as np
import uuid
import os
import shutil
from google.colab import drive
import warnings
warnings.filterwarnings('ignore')

print("🚀 INICIANDO PROCESO COMPLETO...")

# 1. DESCARGAR BASE DE KAGGLE
print("📥 Descargando base...")
path = kagglehub.dataset_download("pradeepkumar2424/usa-banking-transactions-dataset-2023-2024")
csv_file = [f for f in os.listdir(path) if f.endswith('.csv')][0]
df_origen = pd.read_csv(os.path.join(path, csv_file))

# 2. GENERAR 50,000 FILAS CON CRECIMIENTO REALISTA (SIN ZIG-ZAG)
print("🔄 Generando 50,000 filas con tendencia positiva...")
OBJETIVO = 50000
df_generado = df_origen.sample(n=OBJETIVO, replace=True).copy()

# Probabilidades: 2020 (10%) -> 2024 (30%)
years = [2020, 2021, 2022, 2023, 2024]
probs = [0.10, 0.15, 0.20, 0.25, 0.30]
chosen_years = np.random.choice(years, size=OBJETIVO, p=probs)

dates = []
for year, count in pd.Series(chosen_years).value_counts().items():
    start = pd.Timestamp(f"{year}-01-01").value
    end = pd.Timestamp(f"{year}-12-31").value
    dates.extend(pd.to_datetime(np.random.randint(start, end, count)))

df_generado['Transaction_Date'] = np.random.permutation(dates)
df_generado['Transaction_ID'] = [str(uuid.uuid4()) for _ in range(OBJETIVO)]

# 3. ENSUCIAR DATOS (Data Dirtying)
print("🧹 Ensuciando datos (Nulos, Outliers, Texto)...")
df = df_generado.sort_values('Transaction_Date').reset_index(drop=True)
# Nulos
for col in ['Customer_Occupation', 'Transaction_Description', 'City']:
    if col in df.columns: df.loc[df.sample(frac=0.05).index, col] = np.nan
# Outliers en Montos
if 'Transaction_Amount' in df.columns:
    idx = df.sample(n=25).index
    df.loc[idx, 'Transaction_Amount'] *= np.random.uniform(10, 50, 25)

# 4. GUARDAR EN DRIVE
print("💾 Guardando archivo en Google Drive...")

# Montamos el Drive
drive.mount('/content/drive')

# Definimos rutas
nombre_archivo = "dataset_bancario_kdd_50k.csv"
ruta_destino = f"/content/drive/MyDrive/SEXTO SEMESTRE/bifrost/{nombre_archivo}"
carpeta_destino = os.path.dirname(ruta_destino)

# Crear carpeta si no existe y guardar
if not os.path.exists(carpeta_destino):
    os.makedirs(carpeta_destino)

df.to_csv(ruta_destino, index=False)

print("\n" + "="*60)
print("✅ ¡LISTO! ARCHIVO CREADO EXITOSAMENTE")
print("="*60)
print(f"📂 Ubicación: {ruta_destino}")
print(f"📊 Filas: {len(df):,}")
print("📈 Distribución generada (Crecimiento):")
print(df['Transaction_Date'].dt.year.value_counts().sort_index())

🚀 INICIANDO PROCESO COMPLETO...
📥 Descargando base...
Using Colab cache for faster access to the 'usa-banking-transactions-dataset-2023-2024' dataset.
🔄 Generando 50,000 filas con tendencia positiva...
🧹 Ensuciando datos (Nulos, Outliers, Texto)...
💾 Guardando archivo en Google Drive...
Mounted at /content/drive

✅ ¡LISTO! ARCHIVO CREADO EXITOSAMENTE
📂 Ubicación: /content/drive/MyDrive/SEXTO SEMESTRE/bifrost/dataset_bancario_kdd_50k.csv
📊 Filas: 50,000
📈 Distribución generada (Crecimiento):
Transaction_Date
2020     5051
2021     7492
2022     9850
2023    12627
2024    14980
Name: count, dtype: int64


In [1]:
# ═══════════════════════════════════════════════════════════════════════════
# 🕒 GENERADOR DE REPORTES MENSUALES FUTUROS (2023-2025)
# Objetivo: Crear 36 archivos CSV para simular la llegada de datos mes a mes
# ═══════════════════════════════════════════════════════════════════════════

import pandas as pd
import numpy as np
import uuid
import os
import calendar
from google.colab import drive

print("🚀 INICIANDO GENERADOR DE REPORTES MENSUALES...")

# 1. CONECTAR DRIVE
drive.mount('/content/drive')

# 2. CONFIGURACIÓN DE RUTAS
base_path = "/content/drive/MyDrive/SEXTO SEMESTRE/bifrost"
archivo_origen = os.path.join(base_path, "dataset_bancario_kdd_50k.csv")
carpeta_reportes = os.path.join(base_path, "reportes")

# Crear carpeta de reportes si no existe
if not os.path.exists(carpeta_reportes):
    os.makedirs(carpeta_reportes)
    print(f"📂 Carpeta creada: {carpeta_reportes}")
else:
    print(f"📂 Carpeta detectada: {carpeta_reportes}")

# 3. LEER DATASET BASE (Para usarlo como plantilla)
if os.path.exists(archivo_origen):
    print("📖 Leyendo dataset base para copiar estructura...")
    df_base = pd.read_csv(archivo_origen)
else:
    raise FileNotFoundError("❌ No se encuentra el archivo 'dataset_bancario_kdd_50k.csv'. Ejecuta el paso anterior primero.")

# 4. GENERACIÓN DE LOS 36 ARCHIVOS (2023, 2024, 2025)
anios_futuros = [2023, 2024, 2025]
total_archivos = 0

print("\n⚙️ Generando archivos mensuales (Simulación de Crecimiento)...")

for year in anios_futuros:
    for month in range(1, 13): # Meses 1 al 12

        # A. DEFINIR VOLUMEN (Simulamos crecimiento anual)
        # 2023: ~800 txs/mes | 2024: ~1000 txs/mes | 2025: ~1200 txs/mes
        base_volumen = 800 + ((year - 2023) * 200)
        n_filas = np.random.randint(base_volumen, base_volumen + 150)

        # B. TOMAR MUESTRA ALEATORIA (Bootstrapping)
        df_mes = df_base.sample(n=n_filas, replace=True).copy()

        # C. GENERAR FECHAS CORRECTAS PARA ESE MES
        ultimo_dia = calendar.monthrange(year, month)[1]
        dias = np.random.randint(1, ultimo_dia + 1, n_filas)
        horas = np.random.randint(0, 24, n_filas)
        minutos = np.random.randint(0, 60, n_filas)
        seconds = np.random.randint(0, 60, n_filas)

        # Construir columna de fecha
        fechas = pd.to_datetime({
            'year': year, 'month': month, 'day': dias,
            'hour': horas, 'minute': minutos, 'second': seconds
        })

        df_mes['Transaction_Date'] = fechas.sort_values().values

        # D. GENERAR NUEVOS IDs ÚNICOS (Para no repetir los del entrenamiento)
        df_mes['Transaction_ID'] = [str(uuid.uuid4()) for _ in range(n_filas)]

        # E. LEVE INFLACIÓN EN MONTOS (Opcional: Hace más real la predicción)
        # Aumentamos un 3% el monto por cada año que pasa
        factor_inflacion = 1 + (0.03 * (year - 2022))
        if 'Transaction_Amount' in df_mes.columns:
            df_mes['Transaction_Amount'] = df_mes['Transaction_Amount'] * factor_inflacion
            df_mes['Transaction_Amount'] = df_mes['Transaction_Amount'].round(2)

        # F. GUARDAR CSV INDIVIDUAL
        nombre_reporte = f"Reporte_{year}_{month:02d}.csv" # Ej: Reporte_2023_01.csv
        ruta_guardado = os.path.join(carpeta_reportes, nombre_reporte)

        # Seleccionamos solo las columnas originales para mantener limpieza
        cols = df_base.columns
        df_mes[cols].to_csv(ruta_guardado, index=False)

        total_archivos += 1
        # Feedback visual cada 6 meses para no saturar consola
        if month % 6 == 0:
            print(f"   ✅ Generado: {nombre_reporte} ({n_filas} filas)")

print("\n" + "="*60)
print(f"🎉 ¡MISION CUMPLIDA! SE GENERARON {total_archivos} ARCHIVOS CSV.")
print("="*60)
print(f"📂 Ubicación: {carpeta_reportes}")
print("👉 Ahora puedes usar el botón 'Subir Reporte Real' en tu web y seleccionar cualquiera de estos archivos.")

🚀 INICIANDO GENERADOR DE REPORTES MENSUALES...
Mounted at /content/drive
📂 Carpeta creada: /content/drive/MyDrive/SEXTO SEMESTRE/bifrost/reportes
📖 Leyendo dataset base para copiar estructura...

⚙️ Generando archivos mensuales (Simulación de Crecimiento)...
   ✅ Generado: Reporte_2023_06.csv (881 filas)
   ✅ Generado: Reporte_2023_12.csv (924 filas)
   ✅ Generado: Reporte_2024_06.csv (1077 filas)
   ✅ Generado: Reporte_2024_12.csv (1034 filas)
   ✅ Generado: Reporte_2025_06.csv (1238 filas)
   ✅ Generado: Reporte_2025_12.csv (1250 filas)

🎉 ¡MISION CUMPLIDA! SE GENERARON 36 ARCHIVOS CSV.
📂 Ubicación: /content/drive/MyDrive/SEXTO SEMESTRE/bifrost/reportes
👉 Ahora puedes usar el botón 'Subir Reporte Real' en tu web y seleccionar cualquiera de estos archivos.
